In [1]:
import numpy as np

from triqs.gf import Gf, MeshImFreq, Idx
from triqs.gf import inverse, iOmega_n
from triqs.lattice.tight_binding import TBLattice


from triqs_tprf.lattice import lattice_dyson_g0_wk
from triqs_tprf.lattice import lattice_dyson_g_wk

from triqs_tprf.lattice import fourier_wk_to_wr
from triqs_tprf.lattice import chi_wr_from_chi_wk

from triqs_tprf.gw_solver import GWSolver

from hugoDimer import GWHubbardDimer
from casperDimer import GWA


# import sys
# import os
# sys.stdout = open(os.devnull, 'w')

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rcParams['figure.dpi'] = 100

Starting serial run at: 2023-11-07 11:45:58.021022


In [2]:
def polarizationCheck(myPolarization, hugoPolarization):
    np.testing.assert_array_almost_equal(hugoPolarization[:, Idx(0, 0, 0)][0,0,0,0].data, myPolarization['up'].data[:, 0, 0, 0, 0])
    np.testing.assert_array_almost_equal(hugoPolarization[:, Idx(1, 0, 0)][0,0,0,0].data, myPolarization['up'].data[:, 1, 1, 0, 0])
    
def screenedPotentialCheck(myPotential, hugoPotential):
    np.testing.assert_array_almost_equal(hugoPotential[:, Idx(0, 0, 0)][0,0,0,0].data, myPotential['up'].data[:, 0, 0, 0, 0])
    np.testing.assert_array_almost_equal(hugoPotential[:, Idx(1, 0, 0)][0,0,0,0].data, myPotential['up'].data[:, 1, 1, 0, 0])

def selfEnergyCheck(myEnergy, hugoEnergy):
    np.testing.assert_array_almost_equal(hugoEnergy[:, Idx(0, 0, 0)][0,0].data, myEnergy['up'].data[:, 0, 0])
    np.testing.assert_array_almost_equal(hugoEnergy[:, Idx(1, 0, 0)][0,0].data, myEnergy['up'].data[:, 0, 1])


def compareFlags(hartree_flag, fock_flag, spin_flag):
    beta = 20.0
    U = 1.5
    t = 1.0
    nw = 1024
    mu = 0.0
    
    hugo = GWHubbardDimer(
        beta = beta, U = U, 
        t = t, nw = nw, maxiter = 1,
        self_interaction = True, spinless = spin_flag,
        hartree_flag = hartree_flag, fock_flag = fock_flag)
    
    casper = GWA(
        beta = beta, U = U, 
        t = t, nw = nw, spinless = spin_flag, static_flag = False, 
        hartree_flag = hartree_flag, fock_flag = fock_flag)

    print("Hartree flag: {}".format(hartree_flag))
    print("Fock flag: {}".format(fock_flag)) 

    polarizationCheck(casper.P_w, hugo.P_wr)
    print("Polarization check: Passed")
    
    screenedPotentialCheck(casper.W_w, hugo.W_wr)
    print("Screened Potential check: Passed")

    selfEnergyCheck(casper.sigma_w, hugo.sigma_wr)
    print("Self-Energy check: Passed")
    

In [3]:
compareFlags(hartree_flag = False, fock_flag = False, spin_flag = True)


╔╦╗╦═╗╦╔═╗ ╔═╗  
 ║ ╠╦╝║║═╬╗╚═╗  
 ╩ ╩╚═╩╚═╝╚╚═╝  GW
TRIQS: GW solver

beta 20.0
N_w   2048
N_k      2

--> GWSolver.solve_iter
spinless True
 Hartree False
    Fock False
      GW True


Timing:                    incl.     excl.
-------------------------------------------------
Density matrix rho_k:      0.001     0.001   0.3% |
Density matrix rho_r:      0.004     0.002   0.6% |
 Density matrix rho_k:     0.002     0.002   0.8% |
Dyson equation:            0.003     0.003   1.1% |
GW Sigma_dyn:              0.038     0.038  14.3% |-----|
Interaction W_wk:          0.002     0.002   0.6% |
Polarization P_wk:         0.028     0.028  10.3% |---|
Susceptibiltiy chi_wk:     0.003     0.003   1.2% |
Other:                     0.190     0.190  70.7% |---------------------------|
-------------------------------------------------
Total:                               0.269 100.0%

--> GWSolver.calc_real_space
Hartree flag: False
Fock flag: False
Polarization check: Passed
Screened Potential

In [4]:
compareFlags(hartree_flag = True, fock_flag = False, spin_flag = True)


╔╦╗╦═╗╦╔═╗ ╔═╗  
 ║ ╠╦╝║║═╬╗╚═╗  
 ╩ ╩╚═╩╚═╝╚╚═╝  GW
TRIQS: GW solver

beta 20.0
N_w   2048
N_k      2

--> GWSolver.solve_iter
spinless True
 Hartree True
    Fock False
      GW True


Timing:                    incl.     excl.
-------------------------------------------------
Density matrix rho_k:      0.001     0.001   0.3% |
Density matrix rho_r:      0.003     0.001   0.6% |
 Density matrix rho_k:     0.002     0.002   0.7% |
Dyson equation:            0.002     0.002   0.7% |
GW Sigma_dyn:              0.037     0.037  14.1% |-----|
Hartree Sigma_H:           0.001     0.001   0.3% |
Interaction W_wk:          0.001     0.001   0.6% |
Polarization P_wk:         0.026     0.026  10.0% |---|
Susceptibiltiy chi_wk:     0.003     0.003   1.3% ||
Other:                     0.190     0.190  71.5% |----------------------------|
-------------------------------------------------
Total:                               0.265 100.0%

--> GWSolver.calc_real_space
Hartree flag: True
Fock flag:

In [5]:
compareFlags(hartree_flag = False, fock_flag = True, spin_flag = True)


╔╦╗╦═╗╦╔═╗ ╔═╗  
 ║ ╠╦╝║║═╬╗╚═╗  
 ╩ ╩╚═╩╚═╝╚╚═╝  GW
TRIQS: GW solver

beta 20.0
N_w   2048
N_k      2

--> GWSolver.solve_iter
spinless True
 Hartree False
    Fock True
      GW True


Timing:                    incl.     excl.
-------------------------------------------------
Density matrix rho_k:      0.001     0.001   0.3% |
Density matrix rho_r:      0.004     0.002   0.6% |
 Density matrix rho_k:     0.002     0.002   0.8% |
Dyson equation:            0.002     0.002   0.7% |
Fock Sigma_F:              0.001     0.001   0.4% |
GW Sigma_dyn:              0.037     0.037  14.1% |-----|
Interaction W_wk:          0.001     0.001   0.5% |
Polarization P_wk:         0.027     0.027  10.3% |---|
Susceptibiltiy chi_wk:     0.003     0.003   1.3% ||
Other:                     0.189     0.189  71.1% |---------------------------|
-------------------------------------------------
Total:                               0.265 100.0%

--> GWSolver.calc_real_space
Hartree flag: False
Fock flag:

In [6]:
compareFlags(hartree_flag = True, fock_flag = True, spin_flag = True)


╔╦╗╦═╗╦╔═╗ ╔═╗  
 ║ ╠╦╝║║═╬╗╚═╗  
 ╩ ╩╚═╩╚═╝╚╚═╝  GW
TRIQS: GW solver

beta 20.0
N_w   2048
N_k      2

--> GWSolver.solve_iter
spinless True
 Hartree True
    Fock True
      GW True


Timing:                    incl.     excl.
-------------------------------------------------
Density matrix rho_k:      0.001     0.001   0.3% |
Density matrix rho_r:      0.004     0.002   0.6% |
 Density matrix rho_k:     0.002     0.002   0.8% |
Dyson equation:            0.002     0.002   0.7% |
Fock Sigma_F:              0.001     0.001   0.3% |
GW Sigma_dyn:              0.037     0.037  13.9% |-----|
Hartree Sigma_H:           0.001     0.001   0.4% |
Interaction W_wk:          0.001     0.001   0.5% |
Polarization P_wk:         0.027     0.027  10.2% |---|
Susceptibiltiy chi_wk:     0.003     0.003   1.3% ||
Other:                     0.189     0.189  71.0% |---------------------------|
-------------------------------------------------
Total:                               0.267 100.0%

--> GWSo

In [7]:
compareFlags(hartree_flag = True, fock_flag = False, spin_flag = False)


╔╦╗╦═╗╦╔═╗ ╔═╗  
 ║ ╠╦╝║║═╬╗╚═╗  
 ╩ ╩╚═╩╚═╝╚╚═╝  GW
TRIQS: GW solver

beta 20.0
N_w   2048
N_k      2

--> GWSolver.solve_iter
spinless False
 Hartree True
    Fock False
      GW True


Timing:                    incl.     excl.
-------------------------------------------------
Density matrix rho_k:      0.002     0.002   0.5% |
Density matrix rho_r:      0.005     0.002   0.5% |
 Density matrix rho_k:     0.004     0.004   1.0% |
Dyson equation:            0.003     0.003   0.8% |
GW Sigma_dyn:              0.067     0.067  19.6% |-------|
Hartree Sigma_H:           0.001     0.001   0.2% |
Interaction W_wk:          0.014     0.014   3.9% |-|
Polarization P_wk:         0.052     0.052  15.1% |-----|
Susceptibiltiy chi_wk:     0.007     0.007   1.9% ||
Other:                     0.194     0.194  56.5% |----------------------|
-------------------------------------------------
Total:                               0.344 100.0%

--> GWSolver.calc_real_space
Hartree flag: True
Fock flag

In [8]:
compareFlags(hartree_flag = False, fock_flag = True, spin_flag = False)


╔╦╗╦═╗╦╔═╗ ╔═╗  
 ║ ╠╦╝║║═╬╗╚═╗  
 ╩ ╩╚═╩╚═╝╚╚═╝  GW
TRIQS: GW solver

beta 20.0
N_w   2048
N_k      2

--> GWSolver.solve_iter
spinless False
 Hartree False
    Fock True
      GW True


Timing:                    incl.     excl.
-------------------------------------------------
Density matrix rho_k:      0.002     0.002   0.5% |
Density matrix rho_r:      0.005     0.002   0.5% |
 Density matrix rho_k:     0.004     0.004   1.0% |
Dyson equation:            0.003     0.003   0.9% |
Fock Sigma_F:              0.001     0.001   0.2% |
GW Sigma_dyn:              0.067     0.067  19.3% |-------|
Interaction W_wk:          0.014     0.014   4.0% |-|
Polarization P_wk:         0.052     0.052  15.0% |-----|
Susceptibiltiy chi_wk:     0.007     0.007   1.9% ||
Other:                     0.196     0.196  56.8% |----------------------|
-------------------------------------------------
Total:                               0.346 100.0%

--> GWSolver.calc_real_space
Hartree flag: False
Fock fla

In [9]:
compareFlags(hartree_flag = True, fock_flag = True, spin_flag = False)


╔╦╗╦═╗╦╔═╗ ╔═╗  
 ║ ╠╦╝║║═╬╗╚═╗  
 ╩ ╩╚═╩╚═╝╚╚═╝  GW
TRIQS: GW solver

beta 20.0
N_w   2048
N_k      2

--> GWSolver.solve_iter
spinless False
 Hartree True
    Fock True
      GW True


Timing:                    incl.     excl.
-------------------------------------------------
Density matrix rho_k:      0.002     0.002   0.5% |
Density matrix rho_r:      0.005     0.002   0.5% |
 Density matrix rho_k:     0.004     0.004   1.0% |
Dyson equation:            0.003     0.003   0.8% |
Fock Sigma_F:              0.001     0.001   0.2% |
GW Sigma_dyn:              0.067     0.067  19.4% |-------|
Hartree Sigma_H:           0.001     0.001   0.2% |
Interaction W_wk:          0.015     0.015   4.2% |-|
Polarization P_wk:         0.052     0.052  15.0% |-----|
Susceptibiltiy chi_wk:     0.006     0.006   1.9% ||
Other:                     0.195     0.195  56.1% |---------------------|
-------------------------------------------------
Total:                               0.347 100.0%

--> GWS

In [10]:
from time import perf_counter

In [11]:
beta = 100.0
U = 1.5
t = 1.0
nw = 10 * 1024
mu = 0.0

hartree_flag = True
fock_flag = True
spin_flag = False

hugo_start = perf_counter() 

hugo = GWHubbardDimer(
    beta = beta, U = U, 
    t = t, nw = nw, maxiter = 1,
    self_interaction = True, spinless = spin_flag,
    hartree_flag = hartree_flag, fock_flag = fock_flag)
hugo_stop = perf_counter() 


casper_start = perf_counter() 
casper = GWA(
    beta = beta, U = U, 
    t = t, nw = nw, spinless = spin_flag, static_flag = False, 
    hartree_flag = hartree_flag, fock_flag = fock_flag)

casper_stop = perf_counter() 


╔╦╗╦═╗╦╔═╗ ╔═╗  
 ║ ╠╦╝║║═╬╗╚═╗  
 ╩ ╩╚═╩╚═╝╚╚═╝  GW
TRIQS: GW solver

beta 100.0
N_w  20480
N_k      2

--> GWSolver.solve_iter
spinless False
 Hartree True
    Fock True
      GW True


Timing:                    incl.     excl.
-------------------------------------------------
Density matrix rho_k:      0.010     0.010   0.3% |
Density matrix rho_r:      0.023     0.002   0.0% |
 Density matrix rho_k:     0.021     0.021   0.6% |
Dyson equation:            0.029     0.029   0.8% |
Fock Sigma_F:              0.001     0.001   0.0% |
GW Sigma_dyn:              0.777     0.777  21.7% |--------|
Hartree Sigma_H:           0.001     0.001   0.0% |
Interaction W_wk:          0.141     0.141   3.9% |-|
Polarization P_wk:         0.628     0.628  17.6% |------|
Susceptibiltiy chi_wk:     0.066     0.066   1.9% ||
Other:                     1.899     1.899  53.1% |--------------------|
-------------------------------------------------
Total:                               3.576 100.0%

--> G

In [12]:
print(hugo_stop - hugo_start)
print(casper_stop - casper_start)

5.340819362550974
15.699410683475435
